In [2]:
import os

In [3]:
%pwd

'c:\\datascience End to End Projects\\TV sales prediciton\\TV-sales-prediction\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\datascience End to End Projects\\TV sales prediciton\\TV-sales-prediction'

In [3]:
import pandas as pd


In [14]:
df.describe()

,TV,Radio,Newspaper,Sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,15.130500
std,85.854236,14.846809,21.778621,5.283892
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,11.000000
50%,149.750000,22.900000,25.750000,16.000000
75%,218.825000,36.525000,45.100000,19.050000
max,296.400000,49.600000,114.000000,27.000000


In [13]:
df=pd.read_csv("C:\\Users\\mahen\\Downloads\\advertising.csv")

In [6]:
# This is called the entity 
from dataclasses import dataclass # here i imported the dataclass from the dataclasses
from pathlib import Path  # here i imported path from pathlib

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path  # these are variables which are present inside the config.yaml file data_transformation code part and here iam mentioning inside the entity of the class
    data_path: Path

In [7]:
from TV_sales.constants import *
from TV_sales.utils.common import read_yaml, create_directories


In [8]:
# this is same part of the code in every step 
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    # only this part get changes in every step, only defining the get_data_transformation_config get changes according to which step we are performing like 01_data_ingestion,02_data_validation
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,  # here iam returning these 2 varaibles by using this code 
            data_path=config.data_path,
        )

        return data_transformation_config

In [9]:
import os
from TV_sales import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [10]:
# here i defined the component of DataTransformationConfig below
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def feature_scalling(self):
        data = pd.read_csv(self.config.data_path)
        data.drop('Newspaper',axis=1,inplace=True)
        standardized_radio=((data['Radio']-data['Radio'].mean())/data['Radio'].std())
        standardized_tv =((data['TV']-data['TV'].mean())/data['TV'].std())
        data['Radio'] = standardized_radio # replaced those scalled values in the columns of tv and radio
        data['TV'] = standardized_tv

        print(data.head)
        return data # here iam returning my scaled data to save in the object of this Data_transformation class



    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


# here i have defined the tarin_test_split below for performing the train_test_split
    def train_test_spliting(self,scaled_data):
        

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(scaled_data) # this line splits the data into train_test_split

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False) # here it saves the train and test data in csv format inisde the artifacts-> transformation folder
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape) # this logs the information about that how many training and testing samples i have 
        logger.info(test.shape)

        print(train.head())
        print(train.shape)
        print(test.shape)

In [11]:
try:
    config = ConfigurationManager() # here iam initlizing my ConfigurationManager
    data_transformation_config = config.get_data_transformation_config() # and here iam getting my get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config) # here iam passing my data_transformation_config it means iam calling this data_transformation_config
    scaled_data=data_transformation.feature_scalling()
    data_transformation.train_test_spliting(scaled_data) # here performing the train_test_split()
except Exception as e: # this part of code will raise error if anything goes wrong
    raise e

[2024-02-03 15:49:36,564: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-03 15:49:36,567: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-03 15:49:36,572: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-03 15:49:36,574: INFO: common: created directory at: artifacts]
[2024-02-03 15:49:36,576: INFO: common: created directory at: artifacts/data_transformation]
<bound method NDFrame.head of            TV     Radio  Sales
0    0.967425  0.979066   22.1
1   -1.194379  1.080097   10.4
2   -1.512360  1.524637   12.0
3    0.051919  1.214806   16.5
4    0.393196 -0.839507   17.9
..        ...       ...    ...
195 -1.267759 -1.317724    7.6
196 -0.615491 -1.236899   14.0
197  0.348934 -0.940539   14.8
198  1.590574  1.261955   25.5
199  0.990720 -0.987687   18.4

[200 rows x 3 columns]>
[2024-02-03 15:49:36,634: INFO: 324909017: Splited data into training and test sets]
[2024-02-03 15:49:36,635: INFO: 324909017: (150, 3)]
[2024-02-0